# This notebook reports analyses for the following paper: 
## Relating psychiatric symptoms and self-regulation during the COVID-19 crisis

This notebook:

* Import training datasets for each set of variables:'task', 'survey', 'psych', 'crisis', 'health risk outcomes'
* Remove variables from training datasets that are not needed 
* EFA on tasks (N = 386, vars = 113), surveys (N = 386, vars = 61), health risk outcomes (N = 386, vars = 50), psych (N = 497, vars = 207, number of factor is specified based on previous solution from Rouault  et al.,2018 ), crisis (N = 2868, vars = 37)
* Comparison between solution obtained by Eisenberg et al., 2019 and the one here on tasks and surveys
* Evaluate solution obtained (Fit indexes, Variance accounted for, Plot factor correlation, Plot BIC) 
* Cross prediction for tasks and surveys
* Get testing datasets 
* Predict factor scores from solution obtained 
* Remove subjects reporting uncompatible demographics between the first and the second assessment
* Put together file for stats analyses
* Run prediction analysis and plot results


**Import libraries and functions** 

In [1]:
import sys
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from dimensional_structure.results_mv import Results
from selfregulation.utils.utils import (get_info, filter_behav_data,get_behav_data, 
                                        get_admin_data,get_demographics, drop_constant_column, 
                                        remove_variables_from_df, drop_not_common_vars, remove_suffix_row, 
                                        get_predicted_score_data)

from selfregulation.utils.data_preparation_utils import check_age, impute_age, check_demo_vars
from selfregulation.utils.result_utils import (get_loadings, load_results, get_fit_indexes, get_EFA, get_HCA,
                                               get_short_names, 
                                               get_var_of_interest, get_paired_diff)
                                               
from selfregulation.utils.plot_utils import save_figure, plot_correlation

from shutil import copyfile
from os import makedirs, path, remove
import json
import pkg_resources
from glob import glob
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import argparse
import pkg_resources
import random
import time
from cross_results_plots import get_EFA_HCA, plot_corr_heatmap
from dimensional_structure.cross_results_utils import run_cross_prediction
from dimensional_structure.cross_results_plots import plot_cross_within_prediction, plot_BIC
from dimensional_structure.utils import get_factor_groups, residualize_baseline
from dimensional_structure.EFA_plots import (plot_factor_correlation, rename_factor_names,format_short_loadings_names,
                                             plot_short_factor_loading)

from dimensional_structure.prediction_plots import (get_predictions, plot_prediction, visualize_importance, 
                                                    plot_fingerprint)


from dimensional_structure.prediction_utils import run_prediction, run_prediction_local

from rpy2.robjects.packages import importr
import rpy2.robjects

%load_ext rpy2.ipython

Using TensorFlow backend.


## Prepare for EFA on training datasets

In [2]:
run_analysis = True
bootstrap = False
boot_iter = 1000
selected_subsets = ['task', 'survey', 'psych', 'crisis']
verbose = True


In [3]:
basedir=get_info('base_directory')
config_file = path.join(basedir, 'dimensional_structure', 'sro_covid_config.json')
subsets = json.load(open(config_file,'r'))  

## Get training datasets

In [4]:
data={}
for subset in subsets:
    name = subset['name']
    if selected_subsets is not None and name not in selected_subsets:
        continue
    print('*'*80)
    print('IMPORTING TRAINING DATASET:', name)
    print('*'*80)
    if name =='psych':
                
        psych_sro = get_behav_data(file = 'subject_x_items_psych_sro.csv', verbose = True)
        psych_rou = get_behav_data(file = 'subject_x_items_psych_rou.csv', verbose = True)

        psych_sro, sro_colname_drop = drop_constant_column(psych_sro, 'SRO') 
        psych_rou, rou_colname_drop = drop_constant_column(psych_rou, 'ROU') 

        print('Dropping', sro_colname_drop, 'because they had no variability in testing dataset')
        psych_rou = remove_variables_from_df(psych_rou, sro_colname_drop) 
        data[name] = psych_rou
        
        print('Dimensions training dataset', data[name].shape)
        
    elif name == 'crisis': 
        df = get_behav_data(file = 'CRISIS_Adult_April_2020_training_covid_imputed.csv', 
                                    data_subset = 'Data_crisis',  verbose=True)
        
        print('Dropping variables referring to period before COVID-19')
        df = df[df.columns.drop(list(df.filter(regex='pre')))] # remove all vars with pre 
        data[name] = df
        print('Dimensions training dataset', data[name].shape)
    else: 
        
        data[name] = get_behav_data(file = 'meaningful_variables_pt_imputed_train.csv', 
                                    data_subset = 'Data_master', filter_regex=name, verbose=True)
        if name == 'task':
            list_vars_task = 'ravens.score.pt'
            
            print('Dropping', list_vars_task, 'as included as explanatory variable in stats model')
            data[name] = data[name].drop(list_vars_task, axis=1)
            print('Dimensions training dataset', data[name].shape)
            
        if name =='survey': 
            list_vars_survey = ['bis11_survey.Attentional.pt', 'bis11_survey.Motor.pt', 'bis11_survey.Nonplanning.pt']            

            print('Dropping', list_vars_survey ,'as BIS included for Psych')
            data[name]= data[name].drop(list_vars_survey, axis =1)
            print('Dimensions training dataset', data[name].shape)
            
print('*'*80)
print('IMPORTING TRAINING DATASET:', 'health and risk behavior')
print('*'*80)
demo_data = get_demographics(data_subset = 'Data_master')
demo_data_training = demo_data.loc[data['task'].index]

list_vars =['BlackoutFlashbackDrugUse', 'NeglectedFamilyDrugUse', 'MedicalProblemsDueToDrugUse', 
            'leisure_time_activity_survey.activity_level']
print('Dropping', list_vars ,'because either no variability or not included as health risk behavior')
demo_data_training= demo_data_training.drop(list_vars, axis =1)

print('Dimensions training dataset', demo_data_training.shape)


********************************************************************************
IMPORTING TRAINING DATASET: task
********************************************************************************
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: meaningful_variables_pt_imputed_train.csv 
 
Dropping ravens.score.pt as included as explanatory variable in stats model
Dimensions training dataset (386, 113)
********************************************************************************
IMPORTING TRAINING DATASET: survey
********************************************************************************
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: meaningful_variables_pt_imputed_train.csv 
 
Dropping ['bis11_survey.Attentional.pt', 'bis11_survey.Motor.pt', 'bis11_survey.Nonplanning.pt'] as BIS included for Psych
Dimensions training dataset (386, 61)
********************************************************************************
IMPORTING TRAINING DATASET: psych

## Run EFA

In [5]:
demographic_factor_names = ['Mental Health',
                            'Drug Use',
                            'Problem Drinking',
                            'Daily Smoking',
                            'Binge Drinking',
                            'Lifetime Smoking']

datafile = 'Covid_train'

results = None
all_results = None
ID = str(random.getrandbits(16)) 
# create/run results for each subset
for subset in subsets:
    name = subset['name']
    if selected_subsets is not None and name not in selected_subsets:
        continue
    if verbose:
        print('*'*79)
        print('SUBSET: %s' % name.upper())
        print('*'*79)
    if run_analysis == True:
        print('*'*79)
        print('Analyzing Subset: %s' % name)
        # ****************************************************************************
        # Filter Data and check number of variable selected is correct 
        # ****************************************************************************
        print('Dataset imputed: number of variables,', len(data[name].columns),  ', number of subjects,', len(data[name]))
        # ****************************************************************************
        # Load Data
        # ****************************************************************************
        # run dimensional analysis
        start = time.time()
        results = Results(datafile=datafile,
                          file = data[name],
                          file_no_impute= data[name],
                          file_demographics  = demo_data_training, 
                          dist_metric='abscorrelation',
                          name=subset['name'],
                          filter_regex='.',
                          boot_iter=boot_iter,
                          ID=ID,
                          residualize_vars=['Age', 'Sex'])
        results.run_demographic_analysis(verbose=verbose, bootstrap=bootstrap)
        # ****************************************************************************
        # EFA using optimal dimensionality
        # ****************************************************************************
        for rotate in ['oblimin']:
            results.run_EFA_analysis(rotate=rotate, 
                                     verbose=verbose, 
                                     bootstrap=bootstrap)
            results.run_clustering_analysis(rotate=rotate, 
                                            verbose=verbose, 
                                            run_graphs=False)
            if subset['dimensionality'] is not None:
                print(subset['dimensionality'])
                results.EFA.results['num_factors'] = subset['dimensionality']
            c = results.EFA.get_c()
            # name factors
            factor_names = subset.get('%s_factor_names' % rotate, None)
            print(factor_names)
            if factor_names:
                results.EFA.name_factors(factor_names, rotate=rotate)
        ID = results.ID.split('_')[1]
        results.DA.name_factors(demographic_factor_names)
        if verbose: print('Saving Subset: %s' % name)
        id_file = results.save_results()
        # ***************************** saving ****************************************
        # copy latest results and prediction to higher directory
        copyfile(id_file, path.join(path.dirname(results.get_output_dir()), 
                                    '%s_results.pkl' % name))

*******************************************************************************
SUBSET: TASK
*******************************************************************************
*******************************************************************************
Analyzing Subset: task
Dataset imputed: number of variables, 113 , number of subjects, 386
*******************************************************************************
Running demographics
*******************************************************************************
Is the data adequate for factor analysis? Yes
Determining Optimal Dimensionality
Best Components:  {'c_metric-BIC': 6}
Creating Factor Tree
No 6 factor solution computed yet! Computing...
Determining Higher Order Factors
# of components not specified, using BIC determined #
*******************************************************************************
Running EFA, rotate: oblimin
*******************************************************************************
Is the data

## Load all results from EFA on training datasets

In [6]:
all_results  = load_results(datafile = 'Covid_train', verbose = False)

## DONT RUN THIS Check correlation between the original solution and the training solution

In [ ]:
lists = ['task', 'survey']
sizes ={'task':30, 'survey':30}
fonts= {'task':30, 'survey': 15}
loadings_orig={}
loadings_train={}
corr_loadings ={}

for subset in lists:
    print('*'*79)
    print(subset)
    print('*'*79)
    
    # ****************************************************************************
    # Obtain loadings for solution on 522 subjects [original]
    # ****************************************************************************
    loadings_orig[subset] = get_loadings('Complete_02-16-2019', subset = subset)
    
    # ****************************************************************************
    # Obtain loadings for solution on 386 subjects [train]
    # ****************************************************************************
    loadings_train[subset] = get_loadings('Covid_train', subset = subset)
   
    # ****************************************************************************
    # Remove suffixes added by the different preprocessing pipelines 
    # ****************************************************************************
    remove_suffix_row(loadings_orig[subset] , '.logTr')
    remove_suffix_row(loadings_orig[subset] , '.ReflogTr')
    remove_suffix_row(loadings_train[subset], '.pt')
    loadings_train[subset].index = [i.replace('_correctlayout', '') for i in loadings_train[subset].index]
    
    
    # ****************************************************************************
    # Keep only common variables 
    # ****************************************************************************
    common_vars = set.intersection(set(loadings_orig[subset].index), 
                                   set(loadings_train[subset].index))
    
    
    loadings_orig[subset] = drop_not_common_vars(loadings_orig[subset], common_vars = common_vars)
    loadings_train[subset] = drop_not_common_vars(loadings_train[subset], common_vars = common_vars)
    assert list(loadings_orig[subset].index) == list(loadings_train[subset].index), 'order rows does not match'
    
    # ****************************************************************************
    # Add suffix to recognise data set
    # ****************************************************************************
    loadings_orig[subset]   = loadings_orig[subset].add_suffix(', Eisenberg et al., 2019')
    loadings_train[subset]  = loadings_train[subset].add_suffix(', Train dataset')
    
    
    # ****************************************************************************
    # Concatenate data 
    # ****************************************************************************
    temp_merged = pd.concat([loadings_orig[subset], loadings_train[subset]], axis=1)    
    corr_loadings[subset] = temp_merged.corr(method='pearson')
    
    plot_dir = path.dirname(all_results[subset].get_plot_dir())
    plot_correlation(corr_loadings[subset], size = sizes[subset],  
                     subset = subset , plot_dir = plot_dir, fontsize = fonts[subset])
   

## Fit indexes


In [7]:
EFA = {}

for subset in selected_subsets:
    print('*'*79)
    print(subset)
    EFA[subset]     = get_EFA(all_results[subset])
    get_fit_indexes(EFA[subset])
    
print('*'*79)
print("Demographics")
DA  = all_results['task'].DA
get_fit_indexes(DA)
    

*******************************************************************************
task
Number of factors: 4
EFA on N = 386 subjects
Fit indexes:
RMSEA index       RMSEA      lower      upper confidence 
0.06018724 0.05904872 0.06159523 0.90000000 

BIC -20910.23644766232
TLI 0.397542396282727
rms, root mean square of the residuals (RMSA) 0.05963770151357489
crms, df corrected root mean square of the residuals 0.061857394370673315
*******************************************************************************
*******************************************************************************
survey
Number of factors: 8
EFA on N = 386 subjects
Fit indexes:
RMSEA index       RMSEA      lower      upper confidence 
0.06415604 0.06169961 0.06689240 0.90000000 

BIC -4609.316891142318
TLI 0.7815952127363427
rms, root mean square of the residuals (RMSA) 0.0393012492478805
crms, df corrected root mean square of the residuals 0.04542256097339117
*******************************************************

## Get cumulative variance explained 

In [8]:
fa_task    =  EFA['task'].results['factor_tree_Rout_%s' % 'oblimin'][4]
fa_survey  =  EFA['survey'].results['factor_tree_Rout_%s' % 'oblimin'][8]
fa_psych   =  EFA['psych'].results['factor_tree_Rout_%s' % 'oblimin'][3]
fa_crisis  =  EFA['crisis'].results['factor_tree_Rout_%s' % 'oblimin'][10]
fa_demo    = DA.results['factor_tree_Rout_%s' % 'oblimin'][6]

In [9]:
%%R -i fa_task -i fa_survey -i fa_psych  -i fa_crisis -i fa_demo
print('Task ***************************************************************')
print(fa_task$Vaccounted)
print('Survey ***************************************************************')
print(fa_survey$Vaccounted)
print('Psych ***************************************************************')
print(fa_psych$Vaccounted)
print('Crisis ***************************************************************')
print(fa_crisis$Vaccounted)
print('Demo ***************************************************************')
print(fa_demo$Vaccounted)

[1] "Task ***************************************************************"
                             ML1        ML3        ML2        ML4
SS loadings           9.17150874 5.59693036 4.38650600 4.41732084
Proportion Var        0.08116379 0.04953036 0.03881864 0.03909133
Cumulative Var        0.08116379 0.13069415 0.16951279 0.20860412
Proportion Explained  0.38908049 0.23743710 0.18608758 0.18739483
Cumulative Proportion 0.38908049 0.62651758 0.81260517 1.00000000
[1] "Survey ***************************************************************"
                            ML1        ML2        ML4        ML8        ML3
SS loadings           7.0276741 5.17681566 4.70950562 3.87847627 3.02238315
Proportion Var        0.1152078 0.08486583 0.07720501 0.06358158 0.04954726
Cumulative Var        0.1152078 0.20007360 0.27727861 0.34086019 0.39040746
Proportion Explained  0.2261157 0.16656426 0.15152854 0.12479014 0.09724530
Cumulative Proportion 0.2261157 0.39267997 0.54420851 0.66899864 0.76624

## Plot factor correlation

In [10]:
for subset in selected_subsets:
    c= EFA[subset].get_c()
    plot_dir = path.dirname(all_results[subset].get_plot_dir())
    plot_factor_correlation(all_results[subset],c, plot_dir = plot_dir, title=False)
    
c= DA.get_c()  
plot_factor_correlation(all_results[subset],c, plot_dir = plot_dir, title=False, DA = True)


## Plot summary of factor loadings

In [11]:
for subset in selected_subsets:
    c= EFA[subset].get_c()
    plot_dir = path.dirname(all_results[subset].get_plot_dir())
    plot_short_factor_loading(all_results[subset],c, subset, plot_dir = plot_dir)


## Plot BIC

In [12]:
all_results.pop('psych')        
plot_BIC(all_results, size=20, ext='png', dpi=300, plot_dir = plot_dir)

## Correlation between DV and do cross prediction tasks and surveys from training dataset

In [13]:
all_results.pop('crisis')
plot_dir = path.dirname(all_results['task'].get_plot_dir())


survey_order = get_EFA_HCA(all_results['survey'], EFA= False)['reorder_vec']
task_order   = get_EFA_HCA(all_results['task'], EFA= False)['reorder_vec']

DVs_data = pd.concat([data['task'].iloc[:, task_order], data['survey'].iloc[:, survey_order]], axis =1)
#Plot correlation map
plot_corr_heatmap(DVs_data, survey_order, task_order, size=4.6*1/2, ext='png', dpi=300, plot_dir=plot_dir)
#Run cross prediction
run_cross_prediction(all_results, save = True)
#Plot cross prediction
plot_cross_within_prediction(path.join(path.dirname(all_results['task'].get_output_dir()), 'cross_prediction.pkl'), 
                                     size=4.6*1/2, ext='png', dpi=300, plot_dir=plot_dir)

## Get testing datasets


In [14]:
master_test ={}
covid_test ={}

for subset in subsets:
    name = subset['name']
    if selected_subsets is not None and name not in selected_subsets:
        continue
    print('*'*80)
    print('IMPORTING TESTING DATASET:', name)
    print('*'*80)
    if name =='psych':
                
        psych_sro = get_behav_data(file = 'subject_x_items_psych_sro.csv', verbose = True)
        psych_sro, sro_colname_drop = drop_constant_column(psych_sro, 'SRO') 
        print('Dropping', sro_colname_drop, 'because they had no variability in testing dataset')
        
        covid_test[name] = psych_sro
        print('Post COVID-19 # Subjs:', len(covid_test[name].index), ';  # DVs:' , len(covid_test[name].columns), '\n')
        
    elif name == 'crisis':
        crisis_sro = get_behav_data(file = 'subject_x_items_crisis_all_surveys.csv', verbose = True)
        cols_crisis_training= data['crisis'].columns
        crisis_sro = crisis_sro[cols_crisis_training]
              
        covid_test[name] = crisis_sro
        print('Post COVID-19,  # Subjs:', len(covid_test[name].index), ';  # DVs:' , len(covid_test[name].columns), '\n')
              
    else:
        master_test[name] = get_behav_data(file = 'meaningful_variables_pt_imputed_test.csv', 
                                       data_subset = 'Data_master', 
                                       filter_regex =name, verbose=True)

        covid_test[name] = get_behav_data(file = 'meaningful_variables_pt_imputed_test.csv', 
                                        data_subset = 'Data', 
                                        filter_regex =name, verbose=True)
        

        if name == 'task':
            print('Dropping', list_vars_task, 'as included as explanatory variable in stats model')
              
            master_test[name] = master_test[name].drop(list_vars_task, axis=1)
            covid_test[name]  = covid_test[name].drop(list_vars_task, axis=1)

            
        if name =='survey': 

            print('Dropping', list_vars_survey ,'as BIS included for Psych')
              
            master_test[name] = master_test[name].drop(list_vars_survey, axis=1)
            covid_test[name]  = covid_test[name].drop(list_vars_survey, axis=1)
              
        print('Pre COVID-19, # Subjs:', len(master_test[name].index), '; # DVs:', len(master_test[name].columns))
        print('Post COVID-19,  # Subjs:', len(covid_test[name].index), ';  # DVs:' , len(covid_test[name].columns), '\n')
        
print('*'*80)
print('IMPORTING TESTING DATASET:', 'demo')
print('*'*80)


#For Health Outcome Measures a bit more work is needed 
loadings = DA.get_loading(DA.get_c(), rotate='oblimin')
list_vars_DA = list(loadings.index)
list_vars_DA.append('Age')
list_vars_DA.append('Sex')


master_test['demo'] = get_behav_data(file = 'demographic_health.csv', data_subset = 'Data_master', verbose=True)
master_test['demo'] = master_test['demo'].loc[master_test['task'].index] #Pick only test subjects 
master_test['demo'] = master_test['demo'][master_test['demo'].columns[master_test['demo'].columns.isin(list_vars_DA)]]
master_test['demo']= residualize_baseline(master_test['demo'] , ['Age', 'Sex']) #Residualize age and gender
print('Pre COVID-19, # Subjs:', len(master_test['demo'].index), '; # DVs:', len(master_test['demo'].columns))


#=========================================================
#Post COVID-9
#=========================================================
covid_test['demo'] = get_behav_data(file = 'demographic_health.csv', data_subset = 'Data', verbose=True)

#*********************************************************
# Quick pre-processing as unit of measures for 
# Longest relationship and Height were different
# for pre and post covid data collection
#*********************************************************
#Longest relationship in months 
covid_test['demo']['LongestRelationship'] = (covid_test['demo']['LongestRelationshipYears']*12) + covid_test['demo']['LongestRelationshipMonths']
#Heigth in hinches
covid_test['demo']['HeightInches'] = (covid_test['demo']['HeightFeet']*12) +  covid_test['demo']['HeightInches']
#Outcome measures are pre and post, keep only post - pre is given by master 
covid_test['demo']=covid_test['demo'].loc[:,~covid_test['demo'].columns.str.contains('_pre')]
#Replace post with empty string to match variables name from eisenberg
covid_test['demo'].columns=covid_test['demo'].columns.str.replace("_post", "")
#*********************************************************
covid_test['demo'] = covid_test['demo'][covid_test['demo'].columns[covid_test['demo'].columns.isin(list_vars_DA)]]
covid_test['demo']  = residualize_baseline(covid_test['demo'] , ['Age', 'Sex'])
print('Post COVID-19, # Subjs:', len(covid_test['demo'].index), '; # DVs:', len(covid_test['demo'].columns))


********************************************************************************
IMPORTING TESTING DATASET: task
********************************************************************************
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: meaningful_variables_pt_imputed_test.csv 
 
Getting dataset: /SRO/Data/Complete_Covid_03-26-2021...:
file: meaningful_variables_pt_imputed_test.csv 
 
Dropping ravens.score.pt as included as explanatory variable in stats model
Pre COVID-19, # Subjs: 107 ; # DVs: 113
Post COVID-19,  # Subjs: 107 ;  # DVs: 113 

********************************************************************************
IMPORTING TESTING DATASET: survey
********************************************************************************
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: meaningful_variables_pt_imputed_test.csv 
 
Getting dataset: /SRO/Data/Complete_Covid_03-26-2021...:
file: meaningful_variables_pt_imputed_test.csv 
 
Dropping ['bis11_su

## Predict subject's scores from factor solution on training datasets

In [15]:
output_dir=path.join(get_info('base_directory'),'Results/dimensional_structure/Covid_test')
factor_names = {}
for subset in subsets:
    name = subset['name']
    factor_names[name] = subset.get('%s_factor_names' % 'oblimin', None)

factor_names_task   = factor_names['task']
factor_names_survey = factor_names['survey']
factor_names_crisis = factor_names['crisis']
factor_names_psych  = factor_names['psych']


master_test_task   = master_test['task']
master_test_survey = master_test['survey']
master_test_demo   =  master_test['demo']

covid_test_task    = covid_test['task']
covid_test_survey  = covid_test['survey']
covid_test_demo    =  covid_test['demo']

covid_test_psych   = covid_test['psych']
covid_test_crisis  = covid_test['crisis']

### Tasks

In [16]:
%%R -i fa_task -i factor_names_task -i master_test_task -i covid_test_task -i output_dir

master_test_scores <-  as.data.frame(predict(fa_task, master_test_task))
covid_test_scores  <-  as.data.frame(predict(fa_task, covid_test_task))

colnames(master_test_scores) <- c(factor_names_task)
colnames(covid_test_scores) <-  c(factor_names_task)

write.csv(master_test_scores, file.path(output_dir, "master_test_scores_task.csv"), row.names=TRUE)
write.csv(covid_test_scores, file.path(output_dir,  "covid_test_scores_task.csv"), row.names=TRUE)


### Surveys

In [17]:
%%R -i fa_survey -i factor_names_survey -i master_test_survey -i covid_test_survey -i output_dir

master_test_scores <-  as.data.frame(predict(fa_survey, master_test_survey))
covid_test_scores  <-  as.data.frame(predict(fa_survey, covid_test_survey))

colnames(master_test_scores) <-c(factor_names_survey)
colnames(covid_test_scores)  <-c(factor_names_survey)


write.csv(master_test_scores, file.path(output_dir, "master_test_scores_survey.csv"), row.names=TRUE)
write.csv(covid_test_scores,  file.path(output_dir,  "covid_test_scores_survey.csv"), row.names=TRUE)


### Psych

In [18]:
%%R -i fa_psych -i factor_names_psych -i covid_test_psych -i output_dir

covid_test_scores  <-  as.data.frame(predict(fa_psych, covid_test_psych))
colnames(covid_test_scores)  <-c(factor_names_psych)


write.csv(covid_test_scores,  file.path(output_dir,  "covid_test_scores_psych.csv"), row.names=TRUE)

### Demo

In [19]:
%%R -i fa_demo -i demographic_factor_names  -i master_test_demo -i covid_test_demo -i output_dir

master_test_scores <-  as.data.frame(predict(fa_demo, master_test_demo))
covid_test_scores  <-  as.data.frame(predict(fa_demo, covid_test_demo))

colnames(master_test_scores) <- c(demographic_factor_names)
colnames(covid_test_scores) <-  c(demographic_factor_names)


write.csv(master_test_scores, file.path(output_dir, "master_test_scores_demo.csv"), row.names=TRUE)
write.csv(covid_test_scores, file.path(output_dir,  "covid_test_scores_demo.csv"), row.names=TRUE)

### Crisis

In [20]:
%%R -i fa_crisis -i factor_names_crisis -i covid_test_crisis -i output_dir

covid_test_scores  <-  as.data.frame(predict(fa_crisis, covid_test_crisis))
colnames(covid_test_scores)  <-c(factor_names_crisis)


write.csv(covid_test_scores,  file.path(output_dir,  "covid_test_scores_crisis.csv"), row.names=TRUE)

## Get final dataframes

In [21]:
#At master and covid assessment the same turker was assigned different subjectID . 
#This file stores the conversion. How this file was obtained is in admin folder, storing turker's ID hence not shared
basedir=get_info('base_directory')
name_master_covid = get_admin_data(path.join(basedir, 'dimensional_structure'), 'sro_name_master_covid.json')

In [22]:
dict_import_regex  = {'demo'    : [None , None ], 
                      'iq'     :  ['ravens.score.pt','ravens.score.pt'], 
                      'stress' :  ['stress_pre', 'stress_post'], 
                      'mindset':  ['mindset', 'mindset']}

dict_import_folder = {'demo'    : ['Data_master' , 'Data'], 
                      'iq'      : ['Data_master' , 'Data'], 
                      'stress'  : ['Data' , 'Data'], #pre and post measures of stress collected only at Data timepoint
                      'mindset' : ['Data' , 'Data']} #pre and post measures of mindset collected only at Data timepoint

dict_import_file   = {'demo'    : ['demographics.csv', 'demographics.csv'], 
                      'iq'      : ['meaningful_variables_pt_imputed_test.csv', 'meaningful_variables_pt_imputed_test.csv'], 
                      'stress'  : ['meaningful_variables_stress_mindset_pt_imputed.csv', 'meaningful_variables_stress_mindset_pt_imputed.csv'], 
                      'mindset' : ['meaningful_variables_stress_mindset_pt_imputed.csv', 'meaningful_variables_stress_mindset_pt_imputed.csv'], 
                      'task'    : ['master_test_scores_task.csv', 'covid_test_scores_task.csv'], 
                      'survey'  : ['master_test_scores_survey.csv', 'covid_test_scores_survey.csv'], 
                      'psych'   : ['covid_test_scores_psych.csv', 'covid_test_scores_psych.csv'], #predicted scores for psych only at Data timepoint
                      'crisis'  : ['covid_test_scores_crisis.csv', 'covid_test_scores_crisis.csv']}#predicted scores for crisis only at Data timepoint
                    

In [23]:
final_master ={}
final_covid={}
times = ['Data_master', 'Data']

for i, time in enumerate(times): 
    print('*' *80)
    print(time)
    print('*' *80)
    for name in dict_import_file.keys():
        print(name)
        if name in ['demo', 'iq', 'stress', 'mindset']:  # Set of raw variables, else import predicted scores
            
            if time =='Data_master': 
                final_master[name] = get_behav_data(file         = dict_import_file[name][i], 
                                                    data_subset  = dict_import_folder[name][i], 
                                                    filter_regex = dict_import_regex[name][i], 
                                                    verbose=True)
            elif time == 'Data': 
                final_covid[name] = get_behav_data(file          = dict_import_file[name][i], 
                                                    data_subset  = dict_import_folder[name][i], 
                                                    filter_regex = dict_import_regex[name][i], 
                                                    verbose=True)
        else: 
            if time =='Data_master': 
                final_master[name] = get_predicted_score_data(output_dir,dict_import_file[name][i], verbose = True)
                
            elif time == 'Data': 
                final_covid[name]  = get_predicted_score_data(output_dir,dict_import_file[name][i], verbose = True)

#Remove labels related to time to concatenate later                
final_master['stress'].columns = [col.replace('_pre.pt', '') for col in final_master['stress']]
final_covid['stress'].columns = [col.replace('_post.pt', '') for col in final_covid['stress']]

#Keep only testing subjects, and rename people so that they have the same id as the one given at covid test                
final_master['demo']   = final_master['demo'].loc[list(final_master['task'].index), :]
final_master['demo']   = final_master['demo'].rename(index=name_master_covid)
final_master['iq']     = final_master['iq'].rename(index=name_master_covid)
final_master['task']   = final_master['task'].rename(index=name_master_covid)
final_master['survey'] = final_master['survey'].rename(index=name_master_covid)  

********************************************************************************
Data_master
********************************************************************************
iq
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: meaningful_variables_pt_imputed_test.csv 
 
task
Getting dataset: /SRO/Results/dimensional_structure/Covid_test/master_test_scores_task.csv
survey
Getting dataset: /SRO/Results/dimensional_structure/Covid_test/master_test_scores_survey.csv
crisis
Getting dataset: /SRO/Results/dimensional_structure/Covid_test/covid_test_scores_crisis.csv
mindset
Getting dataset: /SRO/Data/Complete_Covid_03-26-2021...:
file: meaningful_variables_stress_mindset_pt_imputed.csv 
 
stress
Getting dataset: /SRO/Data/Complete_Covid_03-26-2021...:
file: meaningful_variables_stress_mindset_pt_imputed.csv 
 
psych
Getting dataset: /SRO/Results/dimensional_structure/Covid_test/covid_test_scores_psych.csv
demo
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: demog

### Remove subjects with declared information not trustworthy

In [24]:
subjects_inconsistent_age = check_age(final_master['demo']['Age'], final_covid['demo']['Age'], 0, 4)
final_covid = impute_age(subjects_inconsistent_age, final_covid, final_master, 4)
vars_qc = ['HispanicLatino','HighestEducation','DivorceCount', 'RelationshipNumber','TrafficAccidentsLifeCount', 'ArrestedChargedLifeCount']
drop_subjects  = check_demo_vars(vars_qc, subjects_inconsistent_age,final_master, final_covid )

for df in final_master.keys():
    final_master[df].drop(drop_subjects, inplace =True)
    
for df in final_covid.keys():
    final_covid[df].drop(drop_subjects, inplace =True)


# of subjects declaring an age which does not match time elapsed between assessments: 8
['s013', 's021', 's036', 's044', 's070', 's075', 's081', 's115']
Drop these subjects as incosistent on at least one of the other variables:
 ['s013', 's021', 's036', 's070', 's081']


In [25]:
#Concatenate files
master_df = pd.concat([final_master['task'],
                       final_master['survey'],
                       final_master['psych'], 
                       final_master['mindset'], 
                       final_master['crisis'], 
                       final_master['stress'], 
                       final_master['demo'][['Age', 'Sex']], 
                       final_master['iq']], 
                       axis = 1)
master_df['time'] = 'master'

covid_df = pd.concat([final_covid['task'],
                       final_covid['survey'],
                       final_covid['psych'], 
                       final_covid['mindset'], 
                       final_covid['crisis'], 
                       final_covid['stress'], 
                       final_covid['demo'][['Age', 'Sex']], 
                       final_covid['iq']], 
                       axis = 1)
covid_df['time'] = 'covid'
all_df = pd.concat([master_df, covid_df], axis = 0)

#Make name of variables easier to handle
all_df['sid'] = all_df.index
all_df = get_short_names(all_df)

#Remove the 5 subjects to drop from both time points
print(len(all_df))


204


In [26]:
#Get paired difference for variable acquired longitudinally 
data_all_paired_diff = pd.DataFrame([])
list_of_variables = ['SpeededIP', 'StrategicIP', 'Caution', 'PercResp', 'lon', 'pss', 'soc_supp',
                     'GDM', 'SS', 'EMC', 'RS', 'RP',  'ERT', 'SRT', 'AGR']

for var in list_of_variables: 
    df_var_of_interest           = get_var_of_interest(all_df, var)
    paired_diff                  = get_paired_diff(df_var_of_interest, var)    
    data_all_paired_diff         = pd.concat([data_all_paired_diff, paired_diff], ignore_index = False, axis = 1)

#Add variables used as covariates in the model for consistent visualization
covariates = all_df[['Age', 'Sex', 'ravens.score.pt', 'AD', 'CIT', 'SW', 'mindset_stress.pt',  'mindset_pandemic_catastrophe.pt']][all_df['time'] == 'master']
data_all_paired_diff= pd.concat([data_all_paired_diff, covariates], ignore_index = False, axis = 1) 

#Save
data_all_paired_diff.to_csv(path.join(output_dir , 'data_all_paired_diff_final.csv'))
all_df.to_csv(path.join(output_dir , 'data_all_df_final.csv'))

### Prepare data for prediction analysis

In [27]:
raw_data ={}
clustering = {}
target_scores ={}
factor_scores = {}

#======================================================================================================
# Get psych variables, either DVs or factor scores wich are the target scores and remover gender, age and IQ
#======================================================================================================
target_scores['psych_DVs'] = get_behav_data(file = 'meaningful_variables_psych_pt_imputed.csv', 
                                        data_subset = 'Data', filter_regex = '.', verbose= False)
target_scores['psych_DVs'].drop(drop_subjects , inplace = True)

target_scores['psych_fac'] = final_covid['psych']

#Regress out gender, age, and IQ
raven = final_covid['iq']
age   = final_covid['demo']['Age']
sex   = final_covid['demo']['Sex']
for data in target_scores.copy(): 
    print('Removing age, gender, and IQ from:',  data)
    df = pd.concat([sex,age , raven, target_scores[data]],  axis =1)
    new_name= data + '_regressed'
    target_scores[new_name]    = residualize_baseline(df, ['Age', 'Sex', 'ravens.score.pt'])
    print(  '# Subjs:', len(target_scores[data].index), ';    # DVs:' , 
                        len(target_scores[data].columns), '\n')  
    
#======================================================================================================
# Get raw data and factor scores for tasks and surveys
#======================================================================================================
dictionary  = {"Data_master": "master", "Data": "covid"}
directories = ['Data_master', 'Data']
names = ['task', 'survey']

for i, directory in enumerate(directories):     
    for i, name in enumerate(names):         
        name_dataset= name+'_'+ dictionary.get(directory)
        print('Subset of data:', name)
        
        raw_data[name_dataset] = get_behav_data(file = 'meaningful_variables_pt_imputed_test.csv', 
                                        data_subset = directory, filter_regex =name, verbose=True)
        #======================================================================
        #If data master , need to rename subjects with same labels they get for target
        #======================================================================  
        if directory =='Data_master': 
            raw_data[name_dataset] = raw_data[name_dataset].rename(index = name_master_covid)  
            
        #======================================================================
        #Drop subjects who do are not trustworthy
        #======================================================================  
        raw_data[name_dataset].drop(drop_subjects, inplace = True)
            
        #======================================================================
        #Drop DVs not used in EFA for tasks/surveys
        #======================================================================  
        if name =='task': 
            raw_data[name_dataset]    = raw_data[name_dataset].drop(list_vars_task, axis=1)
        
        if name == 'survey': 
            raw_data[name_dataset]   = raw_data[name_dataset].drop(list_vars_survey, axis=1)

        #======================================================================
        #Get Clustering
        #======================================================================      
        results = load_results(datafile = 'Covid_train', name = name, verbose = False)
        EFA = get_EFA(results[name])
        c   = EFA.get_c()
        HCA = get_HCA(results[name])
        clustering[name_dataset]=HCA.results['EFA%s_%s' % (c, 'oblimin')]
        
        raw_data[name_dataset] = raw_data[name_dataset].sort_index()

#Factor scores 
factor_scores['task_master']   = final_master['task'].sort_index()
factor_scores['survey_master'] = final_master['survey'].sort_index()
factor_scores['task_covid']    = final_covid['task'].sort_index()
factor_scores['survey_covid']  = final_covid['survey'].sort_index()


#Make sure that target and features matrixes are in the same order 
for data in factor_scores.keys(): 
    print(data)
    assert np.array_equal(target_scores['psych_fac_regressed'].index , factor_scores[data].index),'dfs rows not in the same order'
    assert np.array_equal(target_scores['psych_DVs_regressed'].index , factor_scores[data].index), 'dfs rows not in the same order'
print('***')
for data in raw_data.keys(): 
    print(data)
    assert np.array_equal(target_scores['psych_fac_regressed'].index , raw_data[data].index),'dfs rows not in the same order'
    assert np.array_equal(target_scores['psych_DVs_regressed'].index , raw_data[data].index), 'dfs rows not in the same order'
    

Removing age, gender, and IQ from: psych_DVs
# Subjs: 102 ;    # DVs: 9 

Removing age, gender, and IQ from: psych_fac
# Subjs: 102 ;    # DVs: 3 

Subset of data: task
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: meaningful_variables_pt_imputed_test.csv 
 
Subset of data: survey
Getting dataset: /SRO/Data_master/Complete_02-16-2019...:
file: meaningful_variables_pt_imputed_test.csv 
 
Subset of data: task
Getting dataset: /SRO/Data/Complete_Covid_03-26-2021...:
file: meaningful_variables_pt_imputed_test.csv 
 
Subset of data: survey
Getting dataset: /SRO/Data/Complete_Covid_03-26-2021...:
file: meaningful_variables_pt_imputed_test.csv 
 
task_covid
task_master
survey_covid
survey_master
***
task_covid
task_master
survey_covid
survey_master


### Run prediction analysis

In [28]:
output_dir=path.join(get_info('base_directory'),'Results/dimensional_structure/Covid_test')
directories = ['Data_master', 'Data']
classifiers = [ 'ridge']
subsets = ['task', 'survey']

In [ ]:
for i, directory in enumerate(directories):     
    for subset in subsets:
        name = subset
        name_dataset= name+'_'+ dictionary.get(directory)
        print('*'*79)
        print('Running prediction: %s' % name_dataset)
        for classifier in classifiers:
            print('Running Classifier: %s' % classifier)
            for rotate in ['oblimin']:
                #PYSCH FACTOR SCORES AS TARGET
                #No shuffle
                run_prediction_local(factor_scores = factor_scores[name_dataset], 
                target_scores = target_scores['psych_fac_regressed'], 
                raw_data      = raw_data[name_dataset], 
                clustering    = clustering[name_dataset],
                name          = name_dataset,                     
                output_dir    = output_dir, 
                classifier    = classifier, rotate=rotate, shuffle=False,  verbose=True)
            
                #Shuffle
                run_prediction_local(factor_scores = factor_scores[name_dataset], 
                target_scores = target_scores['psych_fac_regressed'],
                raw_data      = raw_data[name_dataset], 
                clustering    = clustering[name_dataset],
                name          = name_dataset,
                output_dir    = output_dir, 
                classifier    = classifier, rotate=rotate, shuffle=2,  verbose=True) #2500
            
            
                #PYSCH DVs AS TARGET
                #No shuffle
                run_prediction_local(factor_scores = factor_scores[name_dataset], 
                target_scores = target_scores['psych_DVs_regressed'],
                raw_data      = raw_data[name_dataset], 
                clustering    = clustering[name_dataset],
                name          = name_dataset,                     
                output_dir    = output_dir, 
                classifier    = classifier, predicting_DVs = True, rotate=rotate, shuffle=False,  verbose=True)
                
                #Shuffle
                run_prediction_local(factor_scores = factor_scores[name_dataset], 
                target_scores = target_scores['psych_DVs_regressed'],
                raw_data      = raw_data[name_dataset], 
                clustering    = clustering[name_dataset],
                name          = name_dataset,                     
                output_dir    = output_dir, 
                classifier    = classifier, predicting_DVs = True, rotate=rotate, shuffle=1,  verbose=True) #2500
                
        # ***************************** saving ****************************************
        prediction_dir = path.join(output_dir, 'prediction_outputs' , name_dataset)
        new_dir = path.join(output_dir, 'prediction_outputs/', directory)
        
        if not path.exists(new_dir):
            makedirs(new_dir)
        for classifier in classifiers:
            for change_flag in [False, True]:
                print('Change flag: %s' % change_flag)
                for subset in ['oblimin', 'raw']:
                    prediction_files = glob(path.join(prediction_dir, '*%s*' % classifier))
                    #Filter by change
                    prediction_files = filter(lambda x: ('change' in x) == change_flag, prediction_files)
                    #Filter by rotate
                    prediction_files = filter(lambda x: subset in x, prediction_files)
                    #Sort by creation file 
                    prediction_files = sorted(prediction_files, key = path.getmtime)[-4:]

                    for filey in prediction_files:
                        filename = '_'.join(path.basename(filey).split('_')[:-1])
                        copyfile(filey, path.join(new_dir, '%s_%s.pkl' % (name, filename)))   

### Plot predictions bar

In [29]:
for name in subsets: 
    classifier = 'ridge'
    outcome    = 'factors'
    ext =      'pdf'
    filename = 'EFA_%s_%s_%s_prediction_bar.%s' % (classifier, outcome, name, ext)
    plot_dir = path.join(output_dir, 'prediction_outputs/ Plots/', filename)
    predictions, insample_predictions, shuffled, insample_shuffled= get_predictions(output_dir, name , outcome)
    plot_prediction(predictions, insample_predictions, shuffled, insample_shuffled, set_vars = name, 
                    comparison_label = '95% shuffled prediction', size = 10, filename=plot_dir, 
                    metric ='R2')

### Plot fingerprint

In [30]:
output_dir=path.join(get_info('base_directory'),'Results/dimensional_structure/Covid_test')
directories = ['Data_master', 'Data']
classifiers = [ 'ridge']
subsets = ['task', 'survey']


for name in subsets: 
    for directory in directories:
        classifier = 'ridge'
        outcome    = 'factors'
        ext = 'pdf'
    
        filename = 'EFA_%s_%s_%s_fingerprint.%s' % (classifier, outcome, name, ext)
        plot_dir = path.join(output_dir, 'prediction_outputs/ Plots/', directory, filename)
        plot_fingerprint(output_dir, directory, outcome, name, filename=plot_dir)

    